In [1]:
import random
import numpy as np
import sys

In [2]:
class nQueens:
    
    def __init__(self,chrom_length=8):
        '''
        Initializes the board state with a board of chrom_length x chrom_length,
        where chrom_length is the length of the chromosome.
        
        Keyword Arguments:
        chrom_length - length of the chromosome; default is 8 as this is aimed at 
                       solving the 8 queen problem
        '''
        self.chrom_length = chrom_length
      
    
    def make_chrom(self):
        '''
        Uses the chrom_length to create a chromosome.
        '''
        chrom = [random.randint(0,self.chrom_length-1) for i in range(self.chrom_length)]
        #chrom = random.sample(range(self.chrom_length), self.chrom_length)
        return chrom
    
    
    def initial_pop(self,pop_size):
        '''
        Creates a population of chromosomes of length chrom_length.
        '''
        population =[self.make_chrom() for i in range(pop_size)]
        return population
    
    
    def chrom_score(self, chrom):
        '''
        Determines the "score" of the chromosome based on the number of horizontal,
        vertical, and diagonal conflicts of the genes in the chromosome. A lower score is
        better as it means less conflicts.
        '''
        conflicts = 0

        conflicts += abs(len(chrom) - len(np.unique(chrom)))
            
        for i in range(len(chrom)):
            for j in range(len(chrom)):
                if i != j:
                    x = abs(i-j)
                    y = (abs(chrom[i] - chrom[j]))
                    if x == y:
                        conflicts += 1
        return conflicts
    
    
    def pop_score(self, population, verbose = True):
        '''
        Expands chrom_score to an entire population of chromosomes.
        
        Keyword Arguments:
        population - list containing the population of chromosomes to score.
        verbose - allows for tracking of total score, average score, and best score
                  of a population; default is true
        
        Returns:
        score_list - list containing the score for each chromosome in the population
        best - the best score of a chromosome in the population
        '''
        score_list = []
        for chrom in population:
            score_list.append(self.chrom_score(chrom))
            
        tot_score = sum(score_list)
        ave_score = tot_score/len(score_list)
        best = min(score_list)
        if verbose == True:
            print('The total score is {}.\nThe average score is {}.\nThe best score is {}'
                  .format(tot_score, ave_score, best))
            
        return score_list, best
    
    
    def sort_population(self, population, score_list):
        '''
        Sorts the population based on scores; best score to worst score.
        '''
        sort_pop = [x for _,x in sorted(zip(score_list,population))]
        return sort_pop
      
        
    def cross_over(self, chrom1, chrom2):  
        '''
        Performs crossover between two chromosomes at a random crossing point.
        Returns the best child.
        '''
        cross_point = random.randint(1,len(chrom1))
        child1 = chrom1[:cross_point] + chrom2[cross_point:]
        child2 = chrom2[:cross_point] + chrom1[cross_point:]

        if self.chrom_score(child1) < self.chrom_score(child2):
            return child1
        elif self.chrom_score(child1) > self.chrom_score(child2):
            return child2
        else:
            choice = random.randint(1,2)
            if choice == 1:
                return child1
            elif choice == 2: 
                return child2
         
        
    def shuffle_mutate(self, chrom):
        '''
        Mutation that shuffles the entire chromosome.
        '''
        mutation = random.shuffle(chrom)
        return mutation
    
    def swap_mutate(self, chrom):
        '''
        Mutation that swaps two random genes within a chromosome.
        '''
        swap = random.sample(range(self.chrom_length), 2)
        chrom[swap[0]], chrom[swap[1]] = chrom[swap[1]], chrom[swap[0]]
        return chrom
    
    def replace_mutate(self, chrom):
        '''
        Mutation that replaces a random gene with a new random number.
        '''
        gene = random.randint(0,self.chrom_length-1)
        chrom[gene] = random.randint(0,self.chrom_length-1)
        return chrom
    
    
    def new_pop(self, population, score_list, swap_rate = 0.01, replace_rate = 0.01, shuffle_rate = 0.001):
        '''
        Creates the next generation by "breeding" the current population, pairing the best with the worst 
        to ensure variation among the children. Keeps the best half of the current population and combines it 
        with the children to create the next generation.
        
        Keyword Arguments:
        population - list containing the current population
        score_list - list containing the scores for the current population
        swap_rate - rate at which the swap mutation occurs; default 0.01 (1%)
        replace_rate - rate at which the replace mutation occurs; default 0.01 (1%)
        shuffle_rate - rate at which the shuffle mutation occurs; default 0.001 (0.1%)
        
        Returns:
        tot_pop - new population containing the best half of the parents and the 
                  best half of the children
        '''
        mutations = 0
        sort_pop = self.sort_population(population, score_list)
        new_gen = []
        for i in range(0, len(sort_pop), 2):
            
            new_gen.append(self.cross_over(sort_pop[i],sort_pop[-i]))
        
        tot_pop = new_gen + sort_pop[:int(len(sort_pop)/2)]
        
        if random.random() < swap_rate:
            mutations += 1
            for chrom in tot_pop:
                chrom = self.swap_mutate(chrom)
                
        if random.random() < replace_rate:
            mutations += 1
            for chrom in tot_pop:
                chrom = self.replace_mutate(chrom)
        
        if random.random() < shuffle_rate:
            mutations += 1
            for chrom in tot_pop:
                chrom = self.shuffle_mutate(chrom)
        
        return tot_pop

In [3]:
def main(chrom_length, swap_rate, replace_rate, shuffle_rate, initial_pop_size, verbose):
    
    game = nQueens(chrom_length = chrom_length)
    population = game.initial_pop(pop_size = initial_pop_size)
    score_list, best = game.pop_score(population, verbose=True)
    
    done = False
    epoch = 0

    while not done:
        print('Epoch:',epoch)
        population = game.new_pop(population, score_list, swap_rate = swap_rate, replace_rate = replace_rate, 
                                  shuffle_rate = shuffle_rate)
        score_list, best = game.pop_score(population, verbose = verbose)
        epoch += 1
        if best == 0:
            print('Done')
            sort_final = game.sort_population(population, score_list)
            done = True
        if epoch%100==0:
            shuffle_rate+=0.001
            swap_rate+=0.01
            replace_rate+=0.01
    
    print('\nNumber of epochs to find a solution:',epoch)
    print('A solution to the {} queen problem is:{}'.format(chrom_length, sort_final[0]))
            
    board = []
    for i in range(chrom_length):
        board.append([""] * chrom_length)
        board[i][sort_final[0][i]] = "Q"
        
    sys.stdout.write("Solution:\n")
    for j in range(chrom_length):
        for i in range(chrom_length):
            if board[i][j] == "Q":
                sys.stdout.write("Q ")
            else:
                sys.stdout.write("_ ")

        sys.stdout.write("\n")

In [4]:
if __name__ == '__main__':
    chrom_length = 8
    swap_rate = 0.01
    replace_rate = 0.01
    shuffle_rate = 0.001
    initial_pop_size = 100
    verbose = True
    main(chrom_length, swap_rate, replace_rate, shuffle_rate, initial_pop_size, verbose)

The total score is 1105.
The average score is 11.05.
The best score is 3
Epoch: 0
The total score is 839.
The average score is 8.39.
The best score is 3
Epoch: 1
The total score is 695.
The average score is 6.95.
The best score is 3
Epoch: 2
The total score is 616.
The average score is 6.16.
The best score is 3
Epoch: 3
The total score is 542.
The average score is 5.42.
The best score is 2
Epoch: 4
The total score is 498.
The average score is 4.98.
The best score is 2
Epoch: 5
The total score is 440.
The average score is 4.4.
The best score is 2
Epoch: 6
The total score is 428.
The average score is 4.28.
The best score is 2
Epoch: 7
The total score is 392.
The average score is 3.92.
The best score is 2
Epoch: 8
The total score is 358.
The average score is 3.58.
The best score is 2
Epoch: 9
The total score is 332.
The average score is 3.32.
The best score is 2
Epoch: 10
The total score is 334.
The average score is 3.34.
The best score is 2
Epoch: 11
The total score is 291.
The average s

The total score is 359.
The average score is 3.59.
The best score is 1
Epoch: 112
The total score is 212.
The average score is 2.12.
The best score is 1
Epoch: 113
The total score is 120.
The average score is 1.2.
The best score is 1
Epoch: 114
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 115
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 116
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 117
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 118
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 119
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 120
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 121
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 122
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 123
The total score is 100.
Th

The best score is 1
Epoch: 219
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 220
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 221
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 222
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 223
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 224
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 225
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 226
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 227
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 228
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 229
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 230
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 2

The total score is 385.
The average score is 3.85.
The best score is 1
Epoch: 331
The total score is 289.
The average score is 2.89.
The best score is 1
Epoch: 332
The total score is 209.
The average score is 2.09.
The best score is 1
Epoch: 333
The total score is 146.
The average score is 1.46.
The best score is 1
Epoch: 334
The total score is 120.
The average score is 1.2.
The best score is 1
Epoch: 335
The total score is 372.
The average score is 3.72.
The best score is 1
Epoch: 336
The total score is 476.
The average score is 4.76.
The best score is 1
Epoch: 337
The total score is 298.
The average score is 2.98.
The best score is 1
Epoch: 338
The total score is 184.
The average score is 1.84.
The best score is 1
Epoch: 339
The total score is 112.
The average score is 1.12.
The best score is 1
Epoch: 340
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 341
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 342
The total score is 

The best score is 1
Epoch: 437
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 438
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 439
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 440
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 441
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 442
The total score is 100.
The average score is 1.0.
The best score is 1
Epoch: 443
The total score is 588.
The average score is 5.88.
The best score is 1
Epoch: 444
The total score is 400.
The average score is 4.0.
The best score is 1
Epoch: 445
The total score is 268.
The average score is 2.68.
The best score is 1
Epoch: 446
The total score is 455.
The average score is 4.55.
The best score is 1
Epoch: 447
The total score is 271.
The average score is 2.71.
The best score is 1
Epoch: 448
The total score is 180.
The average score is 1.8.
The best score is 1
Epoc